In [0]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [0]:
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

In [0]:
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [0]:
baseMapNum = 32
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), 
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)       

In [0]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [17]:
#training
batch_size = 64
epochs=100
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=3*epochs,
                    verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep75.h5')

Epoch 1/300
781/781 [==============================] - 28s 36ms/step - loss: 1.9470 - accuracy: 0.4269 - val_loss: 1.3312 - val_accuracy: 0.5820
Epoch 2/300
781/781 [==============================] - 27s 35ms/step - loss: 1.3848 - accuracy: 0.5827 - val_loss: 1.0299 - val_accuracy: 0.6719
Epoch 3/300
781/781 [==============================] - 27s 34ms/step - loss: 1.1635 - accuracy: 0.6436 - val_loss: 1.0732 - val_accuracy: 0.6845
Epoch 4/300
781/781 [==============================] - 27s 34ms/step - loss: 1.0419 - accuracy: 0.6792 - val_loss: 0.9009 - val_accuracy: 0.7316
Epoch 5/300
781/781 [==============================] - 27s 34ms/step - loss: 0.9607 - accuracy: 0.7058 - val_loss: 0.8773 - val_accuracy: 0.7429
Epoch 6/300
781/781 [==============================] - 27s 34ms/step - loss: 0.9007 - accuracy: 0.7258 - val_loss: 0.7868 - val_accuracy: 0.7686
Epoch 7/300
781/781 [==============================] - 27s 34ms/step - loss: 0.8624 - accuracy: 0.7410 - val_loss: 0.8013 - val_ac

In [21]:
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('Result with 300 epochs, lr=0.001, decay=1e-6 :')
print('Test loss', scores[0])
print('Test accuracy', scores[1]*100)

10000/10000 [==============================] - 1s 64us/step
Test loss 0.46630742025375366
Test accuracy 88.41999769210815


In [18]:
opt_rms = keras.optimizers.rmsprop(lr=0.0005,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep100.h5')

Epoch 1/100
781/781 [==============================] - 27s 34ms/step - loss: 0.4843 - accuracy: 0.8843 - val_loss: 0.4774 - val_accuracy: 0.8889
Epoch 2/100
781/781 [==============================] - 26s 34ms/step - loss: 0.4686 - accuracy: 0.8884 - val_loss: 0.5312 - val_accuracy: 0.8766
Epoch 3/100
781/781 [==============================] - 26s 33ms/step - loss: 0.4658 - accuracy: 0.8904 - val_loss: 0.5202 - val_accuracy: 0.8795
Epoch 4/100
781/781 [==============================] - 26s 33ms/step - loss: 0.4665 - accuracy: 0.8898 - val_loss: 0.4843 - val_accuracy: 0.8863
Epoch 5/100
781/781 [==============================] - 26s 33ms/step - loss: 0.4560 - accuracy: 0.8907 - val_loss: 0.4800 - val_accuracy: 0.8864
Epoch 6/100
781/781 [==============================] - 26s 33ms/step - loss: 0.4545 - accuracy: 0.8925 - val_loss: 0.5372 - val_accuracy: 0.8760
Epoch 7/100
781/781 [==============================] - 26s 34ms/step - loss: 0.4534 - accuracy: 0.8895 - val_loss: 0.4979 - val_ac

In [24]:
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('Result with 100 epochs, lr=0.0005, decay=1e-6 :')
print('Test loss', scores[0])
print('Test accuracy', scores[1]*100)

10000/10000 [==============================] - 1s 69us/step
Result with 100 epochs, lr=0.0005, decay=1e-6 :
Test loss 0.46630742025375366
Test accuracy 88.41999769210815


In [19]:
opt_rms = keras.optimizers.rmsprop(lr=0.0003,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep125.h5')

Epoch 1/100
781/781 [==============================] - 27s 34ms/step - loss: 0.3932 - accuracy: 0.9026 - val_loss: 0.4687 - val_accuracy: 0.8893
Epoch 2/100
781/781 [==============================] - 26s 33ms/step - loss: 0.3897 - accuracy: 0.9026 - val_loss: 0.4585 - val_accuracy: 0.8874
Epoch 3/100
781/781 [==============================] - 26s 33ms/step - loss: 0.3845 - accuracy: 0.9060 - val_loss: 0.4811 - val_accuracy: 0.8826
Epoch 4/100
781/781 [==============================] - 26s 33ms/step - loss: 0.3838 - accuracy: 0.9068 - val_loss: 0.4648 - val_accuracy: 0.8861
Epoch 5/100
781/781 [==============================] - 26s 33ms/step - loss: 0.3827 - accuracy: 0.9043 - val_loss: 0.4655 - val_accuracy: 0.8862
Epoch 6/100
781/781 [==============================] - 26s 33ms/step - loss: 0.3829 - accuracy: 0.9067 - val_loss: 0.4688 - val_accuracy: 0.8895
Epoch 7/100
781/781 [==============================] - 26s 33ms/step - loss: 0.3769 - accuracy: 0.9048 - val_loss: 0.4480 - val_ac

In [23]:
#testing - no kaggle eval
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('Result with 100 epochs, lr=0.0003, decay=1e-6 : ')
print('Test loss', scores[0])
print('Test accuracy', scores[1]*100)

10000/10000 [==============================] - 1s 66us/step
Result with 100 epochs, lr=0.0003, decay=1e-6 : 
Test loss 0.46630742025375366
Test accuracy 88.41999769210815
